In [ ]:
%matplotlib inline
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Transfer Learning

In this exercise we will implement some important parts of `Two-stage TrAdaBoost`.

For more information check David Pardoe et al. 2010, *Boosting for Regression Transfer*, ICML 2010.

We will implement three classes: `Stage2_TrAdaBoostR2` contains only Stage 2 of our Algorithm. `TwoStageTrAdaBoostR2` contains both Stages, using the class `Stage2_TrAdaBoostR2`.

They both need boosting, a function provided by the `TrAdaBoostR2Base` class.

In [ ]:
class TrAdaBoostR2Base:
    def __init__(self,
                 base_estimator = DecisionTreeRegressor(max_depth=4),
                 sample_size = None,
                 n_estimators = 50,
                 learning_rate = 1.,
                 loss = 'linear',
                 random_state = np.random.mtrand._rand):
        self.base_estimator = base_estimator
        self.sample_size = sample_size
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.loss = loss
        self.random_state = random_state
        
    def boosting(self, X, y, sample_weight):
        # first, we make a copy of our estimator
        ####################
        # Your Code Here   #
        ####################
                
        # we use weighted sampling of the training set with replacement
        # that means we want to have an equally sized training set
        # but with some of them replaced by a copy
        # not uniformly distributed but accounting the sample weights
        # the higher its weight, the more often it is chosen as training sample
        
        ####################
        # Your Code Here   #
        ####################

        # now we fit the chosen training sample set and get a prediction of X
        ####################
        # Your Code Here   #
        ####################
        
        # calculate the absolute error vector of our prediction
        ####################
        # Your Code Here   #
        ####################
        
        # normalize the error vector
        ####################
        # Your Code Here   #
        ####################
        
        # calculate the error according to loss function
        ####################
        # Your Code Here   #
        ####################
        return estimator, error_vect

In [ ]:
class Stage2_TrAdaBoostR2(TrAdaBoostR2Base):
    def __init__(self,
                 base_estimator = DecisionTreeRegressor(max_depth=4),
                 sample_size = None,
                 n_estimators = 50,
                 learning_rate = 1.,
                 loss = 'linear',
                 random_state = np.random.mtrand._rand):
        super().__init__(base_estimator, sample_size, n_estimators, learning_rate, loss, random_state)


    def fit(self, X, y, sample_weight=None):
        # Check parameters
        if self.learning_rate <= 0:
            raise ValueError("learning_rate must be greater than zero")

        if sample_weight is None:
            # Initialize weights to 1 / n_samples
            sample_weight = np.empty(X.shape[0], dtype=np.float64)
            sample_weight[:] = 1. / X.shape[0]
        else:
            # Normalize existing weights
            sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

            # Check that the sample weights sum is positive
            if sample_weight.sum() <= 0:
                raise ValueError(
                      "Attempting to fit with a non-positive "
                      "weighted number of samples.")

        if self.sample_size is None:
            raise ValueError("Additional input required: sample size of source and target is missing")
        elif np.array(self.sample_size).sum() != X.shape[0]:
            raise ValueError("Input error: the specified sample size does not equal to the input size")

        # Clear any previous fit results
        self.estimators_ = []
        self.estimator_weights_ = np.zeros(self.n_estimators, dtype=np.float64)
        self.estimator_errors_ = np.ones(self.n_estimators, dtype=np.float64)

        for iboost in range(self.n_estimators): # this for loop has to be sequencial
            # Boosting step
            sample_weight, estimator_weight, estimator_error = self._stage2_adaboostR2(
                    iboost,
                    X, y,
                    sample_weight)
            # Early termination
            if sample_weight is None:
                break

            self.estimator_weights_[iboost] = estimator_weight
            self.estimator_errors_[iboost] = estimator_error

            # Stop if error is zero
            if estimator_error == 0:
                break

            sample_weight_sum = np.sum(sample_weight)

            # Stop if the sum of sample weights has become non-positive
            if sample_weight_sum <= 0:
                break

            if iboost < self.n_estimators - 1:
                # Normalize
                sample_weight /= sample_weight_sum
        return self
                
    def _stage2_adaboostR2(self, iboost, X, y, sample_weight):
        estimator, error_vect = self.boosting(X, y, sample_weight)

        # add the fitted estimator to our estimators list
        ####################
        # Your Code Here   #
        ####################

        # Calculate the weighed accumulated estimator error
        ####################
        # Your Code Here   #
        ####################

        # Stop if fit is perfect
        ####################
        # Your Code Here   #
        ####################
        
        # Remove estimator if estimator error is >= 0.5
        # and if it's not the only one
        ####################
        # Your Code Here   #
        ####################
        
        # calculate the new beta_t
        ####################
        # Your Code Here   #
        ####################

        # avoid overflow of np.log(1. / beta)
        if beta < 1e-308:
            beta = 1e-308
        
        # calculate the new weight for hypothesis h_t
        ####################
        # Your Code Here   #
        ####################

        # update the weight vectors according to AdaBoost.R2 except the weights of the source data
        # use the sum of all sample weights as normalizing constant
        ####################
        # Your Code Here   #
        ####################

        return sample_weight, estimator_weight, estimator_error


    def predict(self, X):
        # Evaluate predictions of all estimators
        ####################
        # Your Code Here   #
        ####################
        
        # Sort the predictions and get indices
        ####################
        # Your Code Here   #
        ####################
        
        # Find index of median prediction for each sample
        ####################
        # Your Code Here   #
        ####################
        
        # Get the median estimators
        ####################
        # Your Code Here   #
        ####################
        
        # Get the median predictions
        ####################
        # Your Code Here   #
        ####################
        
        # Return median predictions
        return median_predictions

Now we want to compare it to `Two-stage TrAdaBoostR2`, which is already implemented:

In [ ]:
class TwoStageTrAdaBoostR2(TrAdaBoostR2Base):
    def __init__(self,
                 base_estimator = DecisionTreeRegressor(max_depth=4),
                 sample_size = None,
                 n_estimators = 50,
                 steps = 10,
                 fold = 5,
                 learning_rate = 1.,
                 loss = 'linear',
                 random_state = np.random.mtrand._rand):
        self.steps = steps
        self.fold = fold
        super().__init__(base_estimator, sample_size, n_estimators, learning_rate, loss, random_state)

    def fit(self, X, y, sample_weight=None):
        # Check parameters
        if self.learning_rate <= 0:
            raise ValueError("learning_rate must be greater than zero")

        if sample_weight is None:
            # Initialize weights to 1 / n_samples
            sample_weight = np.empty(X.shape[0], dtype=np.float64)
            sample_weight[:] = 1. / X.shape[0]
        else:
            # Normalize existing weights
            sample_weight /= sample_weight.sum(dtype=np.float64)

            # Check that the sample weights sum is positive
            if sample_weight.sum() <= 0:
                raise ValueError(
                      "Attempting to fit with a non-positive "
                      "weighted number of samples.")

        if self.sample_size is None:
            raise ValueError("Additional input required: sample size of source and target is missing")
        elif np.array(self.sample_size).sum() != X.shape[0]:
            raise ValueError("Input error: the specified sample size does not equal to the input size")


        X_source = X[:-self.sample_size[-1]]
        y_source = y[:-self.sample_size[-1]]
        X_target = X[-self.sample_size[-1]:]
        y_target = y[-self.sample_size[-1]:]

        self.models_ = []
        self.errors_ = []
        for istep in range(self.steps):
            model = Stage2_TrAdaBoostR2(self.base_estimator,
                                        sample_size = self.sample_size,
                                        n_estimators = self.n_estimators,
                                        learning_rate = self.learning_rate, loss = self.loss,
                                        random_state = self.random_state)
            model.fit(X, y, sample_weight = sample_weight)
            self.models_.append(model)
            # cv training
            kf = KFold(n_splits = self.fold)
            error = []
            target_weight = sample_weight[-self.sample_size[-1]:]
            source_weight = sample_weight[:-self.sample_size[-1]]
            for train, test in kf.split(X_target):
                sample_size = [self.sample_size[0], len(train)]
                model = Stage2_TrAdaBoostR2(self.base_estimator,
                                        sample_size = sample_size,
                                        n_estimators = self.n_estimators,
                                        learning_rate = self.learning_rate, loss = self.loss,
                                        random_state = self.random_state)
                X_train = np.concatenate((X_source, X_target[train]))
                y_train = np.concatenate((y_source, y_target[train]))
                X_test = X_target[test]
                y_test = y_target[test]
                # make sure the sum weight of the target data do not change with CV's split sampling
                target_weight_train = target_weight[train]*np.sum(target_weight)/np.sum(target_weight[train])
                model.fit(X_train, y_train, sample_weight = np.concatenate((source_weight, target_weight_train)))
                y_predict = model.predict(X_test)
                error.append(mean_squared_error(y_predict, y_test))

            self.errors_.append(np.array(error).mean())

            sample_weight = self._twostage_adaboostR2(istep, X, y, sample_weight)

            if sample_weight is None:
                break
            if np.array(error).mean() == 0:
                break

            sample_weight_sum = np.sum(sample_weight)

            # Stop if the sum of sample weights has become non-positive
            if sample_weight_sum <= 0:
                break

            if istep < self.steps - 1:
                # Normalize
                sample_weight /= sample_weight_sum
        return self


    def _twostage_adaboostR2(self, istep, X, y, sample_weight):
        estimator, error_vect = self.boosting(X, y, sample_weight)

        # Update the weight vector
        beta = self._beta_binary_search(istep, sample_weight, error_vect, stp = 1e-30)

        if not istep == self.steps - 1:
            sample_weight[:-self.sample_size[-1]] *= np.power(
                    beta,
                    (error_vect[:-self.sample_size[-1]]) * self.learning_rate)
        return sample_weight


    def _beta_binary_search(self, istep, sample_weight, error_vect, stp):
        # calculate the specified sum of weight for the target data
        n_target = self.sample_size[-1]
        n_source = np.array(self.sample_size).sum() - n_target
        theoretical_sum = n_target/(n_source+n_target) + istep/(self.steps-1)*(1-n_target/(n_source+n_target))
        # for the last iteration step, beta is 0.
        if istep == self.steps - 1:
            beta = 0.
            return beta
        # binary search for beta
        L = 0.
        R = 1.
        beta = (L+R)/2
        sample_weight_ = copy.deepcopy(sample_weight)
        sample_weight_[:-n_target] *= np.power(
                    beta,
                    (error_vect[:-n_target]) * self.learning_rate)
        sample_weight_ /= np.sum(sample_weight_, dtype=np.float64)
        updated_weight_sum = np.sum(sample_weight_[-n_target:], dtype=np.float64)

        while np.abs(updated_weight_sum - theoretical_sum) > 0.01:
            if updated_weight_sum < theoretical_sum:
                R = beta - stp
                if R > L:
                    beta = (L+R)/2
                    sample_weight_ = copy.deepcopy(sample_weight)
                    sample_weight_[:-n_target] *= np.power(
                                beta,
                                (error_vect[:-n_target]) * self.learning_rate)
                    sample_weight_ /= np.sum(sample_weight_, dtype=np.float64)
                    updated_weight_sum = np.sum(sample_weight_[-n_target:], dtype=np.float64)
                else:
                    # binary search goal not met, try to reduce the search inverval in the next iteration
                    break

            elif updated_weight_sum > theoretical_sum:
                L = beta + stp
                if L < R:
                    beta = (L+R)/2
                    sample_weight_ = copy.deepcopy(sample_weight)
                    sample_weight_[:-n_target] *= np.power(
                                beta,
                                (error_vect[:-n_target]) * self.learning_rate)
                    sample_weight_ /= np.sum(sample_weight_, dtype=np.float64)
                    updated_weight_sum = np.sum(sample_weight_[-n_target:], dtype=np.float64)
                else:
                    # binary search goal not met, try to reduce the search inverval in the next iteration
                    break
        return beta


    def predict(self, X):
        # select the model with the least CV error
        fmodel = self.models_[np.array(self.errors_).argmin()]
        predictions = fmodel.predict(X)
        return predictions

What are the differences? 

Now we can test our implementation:

In [ ]:
def response(x, d, random_state):
    """
    x is the input variable
    d controls the simularity of different tasks
    """
    c = []
    factor_weights = [0.1, 0.1, 0.1 , 0.1, 0.05, 0.05]
    
    for weight in factor_weights:
        c.append(np.random.normal(1, weight*d))

    y = c[0]*np.sin(c[2]*x + c[4]).ravel() + c[1]*np.sin(c[3]*6 * x + c[5]).ravel() + random_state.normal(0, 0.1, x.shape[0])
    return y

d = 0.5
signals_count = 5

rng = np.random.RandomState(1)

n_source = []
x_source = []
y_source = []

for i in range(signals_count):
    n_source.append(100)
    x_source.append(np.linspace(0, 6, n_source[i])[:, np.newaxis])
    y_source.append(response(x_source[i], d, rng))

n_target_train = 15

x_target_train = np.linspace(0, 6, n_target_train)[:, np.newaxis]
y_target_train = response(x_target_train, d, rng)

n_target_test = 600
x_target_test = np.linspace(0, 6, n_target_test)[:, np.newaxis]
y_target_test = response(x_target_test, d, rng)

plt.figure()

for i in range(signals_count):
    plt.plot(x_source[i], y_source[i], label="source"+str(i), linewidth=1)

plt.plot(x_target_test, y_target_test, c="b", label="target_test", linewidth=0.5)
plt.scatter(x_target_train, y_target_train, c="k", label="target_train")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Multiple datasets")
plt.legend()
plt.show()

X = copy.deepcopy(x_source[0])
for i in range(1,signals_count):
    X = np.concatenate((X,x_source[i]))

X = np.concatenate((X, x_target_train))

y = copy.deepcopy(y_source[0])
for i in range(1,signals_count):
    y = np.concatenate((y,y_source[i]))

y = np.concatenate((y, y_target_train))

sample_size = [np.sum(n_source), n_target_train]

n_estimators = 100
steps = 10
fold = signals_count
random_state = np.random.RandomState(1)

regr_1 = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                      n_estimators = n_estimators, sample_size = sample_size, 
                      steps = steps, fold = fold, 
                      random_state = random_state)
regr_1.fit(X, y)
y_pred1 = regr_1.predict(x_target_test)

# test AdaBoostR2 without transfer learning
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=6),
                          n_estimators = n_estimators)

regr_2.fit(x_target_train, y_target_train)
y_pred2 = regr_2.predict(x_target_test)

plt.figure()
plt.scatter(x_target_train, y_target_train, c="k", label="target_train")
plt.plot(x_target_test, y_target_test, c="b", label="target_test", linewidth=0.5)
plt.plot(x_target_test, y_pred1, c="r", label="TwoStageTrAdaBoostR2", linewidth=2)
plt.plot(x_target_test, y_pred2, c="y", label="AdaBoostR2", linewidth=2)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Two-stage Transfer Learning vs. Boosted Decision Tree Regression")
plt.legend()
plt.show()

mse_twostageboost = mean_squared_error(y_target_test, y_pred1)   
mse_adaboost = mean_squared_error(y_target_test, y_pred2)

print("MSE of regular AdaboostR2:", mse_adaboost)
print("MSE of TwoStageTrAdaboostR2:", mse_twostageboost)


Try out different values for the variable $d$, i.e. different levels of similarity of our source data. What happens?